In [ ]:
from gpiozero import LightSensor
import RPi.GPIO as GPIO

In [ ]:
# Stepper motor ULN2003 IN1-IN4
IN1 = 17
IN2 = 18
IN3 = 27
IN4 = 22

# Stepper motor steps for opening shield
steps_open = 128

# Stepper motor steps for closing shield
steps_close = -128

In [ ]:
# Stepper motor half-step sequence
SEQ = [
    [1,0,0,0],
    [1,1,0,0],
    [0,1,0,0],
    [0,1,1,0],
    [0,0,1,0],
    [0,0,1,1],
    [0,0,0,1],
    [1,0,0,1]
]

# Streaks
fog_streak = 0
sun_streak = 0

# Light Sensor
ldr = LightSensor(4)

In [ ]:
def step_motor(steps, delay=0.002):
    """Move motor by number of steps"""
    if steps > 0:
        seq = SEQ
    else:
        seq = SEQ[::-1]
        steps = -steps
    for a in range(steps):
        for pattern in seq:
            GPIO.output(IN1, pattern[0])
            GPIO.output(IN2, pattern[1])
            GPIO.output(IN3, pattern[2])
            GPIO.output(IN4, pattern[3])
            time.sleep(delay)

In [ ]:
def fog_or_sun(ldr):
    """Returns a string for the type of weather(foggy, sunny)"""
    if ldr.value < 200:
        return "sun"
    else:
        return "fog"

In [ ]:
def motor_on_weather(ldr):
    weather = fog_or_sun(ldr)
    if weather == "fog":
        if fog_streak == 0:
            step_motor(steps_open)
        fog_streak += 1
        sun_streak = 0
        print("Fog Detected!")
    else:
        if fog_streak >= 1:
            step_motor(steps_close)
        sun_streak += 1
        fog_streak = 0
        print("Sunlight Detected!")

In [ ]:
try:
    while True:
        motor_on_weather(ldr)
except KeyboardInterrupt:
    print("Exiting...")
finally:
    motor_cleanup()